In [ ]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
medals_data = pd.read_csv('medals.csv')

print("Medals Data:-\n")
print(medals_data.isnull().sum())

Medals Data:-

discipline_title             0
slug_game                    0
event_title                  0
event_gender                 0
medal_type                   0
participant_type             0
participant_title        15113
athlete_url               4670
athlete_full_name         3624
country_name                 0
country_code              1502
country_3_letter_code        0
dtype: int64


In [ ]:
medals_data.drop('participant_title', inplace=True, axis=1)
medals_data.drop('country_code', inplace=True, axis=1)
medals_data.drop('athlete_url', inplace=True, axis=1)
print(medals_data.isnull().sum())

discipline_title            0
slug_game                   0
event_title                 0
event_gender                0
medal_type                  0
participant_type            0
athlete_full_name        3624
country_name                0
country_3_letter_code       0
dtype: int64


In [ ]:
medals_data.dropna(inplace = True)

print(medals_data.isnull().sum())

discipline_title         0
slug_game                0
event_title              0
event_gender             0
medal_type               0
participant_type         0
athlete_full_name        0
country_name             0
country_3_letter_code    0
dtype: int64


In [ ]:
print(medals_data)

      discipline_title     slug_game                      event_title  \
0              Curling  beijing-2022                    Mixed Doubles   
1              Curling  beijing-2022                    Mixed Doubles   
2              Curling  beijing-2022                    Mixed Doubles   
3              Curling  beijing-2022                    Mixed Doubles   
4              Curling  beijing-2022                    Mixed Doubles   
...                ...           ...                              ...   
21692    Weightlifting   athens-1896  heavyweight - one hand lift men   
21693    Weightlifting   athens-1896  heavyweight - one hand lift men   
21694    Weightlifting   athens-1896  heavyweight - two hand lift men   
21695    Weightlifting   athens-1896  heavyweight - two hand lift men   
21696    Weightlifting   athens-1896  heavyweight - two hand lift men   

      event_gender medal_type participant_type        athlete_full_name  \
0            Mixed       GOLD         GameTeam  

In [ ]:
enc = LabelEncoder()

# Converting alphanumeric to numeric for training data
medals_data['discipline_title'] = enc.fit_transform(medals_data['discipline_title'])
medals_data['slug_game'] = enc.fit_transform(medals_data['slug_game'])
medals_data['event_title'] = enc.fit_transform(medals_data['event_title'])
medals_data['event_gender'] = enc.fit_transform(medals_data['event_gender'])
medals_data['medal_type'] = enc.fit_transform(medals_data['medal_type'])
medals_data['participant_type'] = enc.fit_transform(medals_data['participant_type'])
medals_data['athlete_full_name'] = enc.fit_transform(medals_data['athlete_full_name'])
medals_data['country_name'] = enc.fit_transform(medals_data['country_name'])
medals_data['country_3_letter_code'] = enc.fit_transform(medals_data['country_3_letter_code'])


In [ ]:
print(medals_data)

       discipline_title  slug_game  event_title  event_gender  medal_type  \
0                    16          8          795             1           1   
1                    16          8          795             1           1   
2                    16          8          795             1           2   
3                    16          8          795             1           2   
4                    16          8          795             1           0   
...                 ...        ...          ...           ...         ...   
21692                65          3         1077             0           2   
21693                65          3         1077             0           0   
21694                65          3         1078             0           1   
21695                65          3         1078             0           2   
21696                65          3         1078             0           0   

       participant_type  athlete_full_name  country_name  \
0              

In [ ]:
# app.py

app = Flask(__name__)

# Constants for API integration (Replace with real API details)
API_BASE_URL = 'https://api.olympic-games.com/'
SCHEDULE_ENDPOINT = 'schedule/'
MEDAL_PREDICTION_ENDPOINT = 'medal-prediction/'




# Load and preprocess the provided dataset
df = pd.get_dummies(medals_data, columns=['discipline_title', 'event_title', 'country_name'])

# Prepare data for training the machine learning model
X = df.drop('medal_type', axis=1)
y = df['medal_type']
model = RandomForestClassifier(random_state=42)
model.fit(X, y)




# Helper functions for API integration
# Sample schedule data (Replace with actual API response)
dummy_schedule_data = [
    {
        "event_id": 1,
        "event_title": "Swimming - Men's 100m Freestyle",
        "event_date": "2023-07-28",
        "event_time": "10:00:00",
        "venue": "Aquatics Center",
        "location": "Tokyo, Japan"
    },
    {
        "event_id": 2,
        "event_title": "Athletics - Women's 400m Final",
        "event_date": "2023-07-28",
        "event_time": "18:30:00",
        "venue": "Olympic Stadium",
        "location": "Tokyo, Japan"
    },
    {
        "event_id": 3,
        "event_title": "Basketball - Men's Semifinal",
        "event_date": "2023-07-29",
        "event_time": "14:00:00",
        "venue": "Saitama Super Arena",
        "location": "Saitama, Japan"
    },
    # Add more events here
]

def fetch_schedule():
    # Replace this dummy data with actual API request and response handling
    return dummy_schedule_data

'''
def fetch_schedule():
    try:
        response = requests.get(API_BASE_URL + SCHEDULE_ENDPOINT)
        if response.status_code == 200:
            schedule_data = response.json()
            return schedule_data
    except requests.RequestException as e:
        print(f"Error fetching schedule: {e}")
    return None

    ---------------As we were not able to get an API for live schedule we used dummy data
            but the proper code for a live API is in this docstring---------------
'''





# Sample dataset (Replace with the actual dataset)

df = medals_data

def fetch_medal_prediction(event_id):
    # Filter the dataset for the given event_id
    event_data = df[df['event_id'] == event_id]

    if event_data.empty:
        return None

    # Group the data by country and count the number of each medal type (Gold, Silver, Bronze)
    medal_counts = event_data['medal_type'].value_counts()

    # Get the most common medal (Gold, if tied, take the first one)
    most_common_medal = medal_counts.idxmax()

    # Get the country/countries with the most common medal
    predicted_countries = list(event_data[event_data['medal_type'] == most_common_medal]['country_name'].unique())

    # Prepare the medal prediction data
    medal_prediction_data = {
        "event_id": event_id,
        "event_title": event_data['event_title'].iloc[0],  # Assume the event title is the same for all rows of the event
        "medal": most_common_medal,
        "predicted_countries": predicted_countries
    }

    return medal_prediction_data





# Flask routes for different app sections
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/schedule')
def schedule():
    schedule_data = fetch_schedule()
    return render_template('schedule.html', schedule_data=schedule_data)

@app.route('/predict_medal', methods=['POST'])
def predict_medal():
    if request.method == 'POST':
        event_id = request.form['event_id']
        medal_prediction = fetch_medal_prediction(event_id)
        return render_template('medal_prediction.html', medal_prediction=medal_prediction)





# Sample dataset (Replace with the actual dataset)
df = medals_data

@app.route('/data_insights')
def data_insights():
    # Calculate the total number of medals won by each country
    country_medal_counts = df.groupby('country_name')['medal_type'].value_counts().unstack().fillna(0)

    # Calculate the total number of medals won by each participant
    participant_medal_counts = df.groupby('athlete_full_name')['medal_type'].value_counts().unstack().fillna(0)

    # Get the most successful country (highest total medals)
    most_successful_country = country_medal_counts.sum(axis=1).idxmax()

    # Get the top-performing athlete (highest total medals)
    top_performing_athlete = participant_medal_counts.sum(axis=1).idxmax()

    # Prepare the data insights
    data_insights = {
        "most_successful_country": most_successful_country,
        "top_performing_athlete": top_performing_athlete,
        "country_medal_counts": country_medal_counts.to_dict(),
        "participant_medal_counts": participant_medal_counts.to_dict(),
    }

    return render_template('data_insights.html', data_insights=data_insights)



if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
